In [1]:
import pandas as pd
import glob

# get all the json files in votes_out_2023_07_21
file_names = glob.glob('votes_out_2023_07_21/*.json')
len(file_names)

6910

In [2]:
# monkeypatch json
# monkeypatch using standard python json module

import json

pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pd.json_normalize(simplejson.loads(s))

In [3]:
# read them all as one big dataframe
df = pd.concat([pd.read_json(f) for f in file_names], ignore_index=True)
df.shape

/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, cop

(5896763, 7)

In [4]:
df.head()

,id,voter,created,vp_by_strategy,vp,space.id,proposal.id
0,0x586056a5f0f05b05b674b194b0c4709f85c2b09a5ab3...,0x46e7BC113E1f0748ddd15B49975c9dDD8E348DD2,1648938795,"[554.508218775, 1e-09]",554.508219,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...
1,0x3f0bf0cdbad6378da031424e8c4011626234f6797281...,0x9A0C4a2847A49769a2361c3fA5c9FC19b239BC1A,1648938250,"[40.978659197, 0]",40.978659,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...
2,0x0d8449d02c52710d94165c527498f9a1bba3e28c998c...,0x7934746C0B363c395Ec01e6eEdDC151f21A5d4b4,1648936664,"[9.194e-06, 2000.111080317]",2000.111090,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...
3,QmWGfUi7KdkqN72m3peuHfLJ8WzLXtFWnW7bWkGKrRa9ii,0xa2908B6212A1F093fF94DC2Cd06F5Df7a711A436,1648934039,"[5.9299e-05, 6.71957472]",6.719634,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...
4,0xfd3b093c6b4b89396dba080eeb8c7345a0af3289b7b8...,0xC1a1D956311AE0809e52b5F9873E0db627fb2D9A,1648933753,"[822.970834554, 0]",822.970835,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...


In [5]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename proposal.id to proposal_id
df.rename(columns={'proposal.id': 'proposal_id'}, inplace=True)
# * rename id to vote_id
df.rename(columns={'id': 'vote_id'}, inplace=True)
# * voter -- the id of the person who made the vote, no chg
# * rename created to date -- this should be as a pandas date, not as an epoch
df.rename(columns={'created': 'date'}, inplace=True)
# * rename vp_by_strategy to choice -- this should be a string as this is not always true/false
df.rename(columns={'vp_by_strategy': 'choice'}, inplace=True)
# make string
df['choice'] = df['choice'].astype(str)
# * rename vp to weight -- the vote weight
df.rename(columns={'vp': 'weight'}, inplace=True)
# make int
df['weight'] = df['weight'].astype(int)
df.head()


/opt/homebrew/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0x586056a5f0f05b05b674b194b0c4709f85c2b09a5ab3...,0x46e7BC113E1f0748ddd15B49975c9dDD8E348DD2,1648938795,"[554.508218775, 1e-09]",554,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
1,0x3f0bf0cdbad6378da031424e8c4011626234f6797281...,0x9A0C4a2847A49769a2361c3fA5c9FC19b239BC1A,1648938250,"[40.978659197, 0]",40,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
2,0x0d8449d02c52710d94165c527498f9a1bba3e28c998c...,0x7934746C0B363c395Ec01e6eEdDC151f21A5d4b4,1648936664,"[9.194e-06, 2000.111080317]",2000,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
3,QmWGfUi7KdkqN72m3peuHfLJ8WzLXtFWnW7bWkGKrRa9ii,0xa2908B6212A1F093fF94DC2Cd06F5Df7a711A436,1648934039,"[5.9299e-05, 6.71957472]",6,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
4,0xfd3b093c6b4b89396dba080eeb8c7345a0af3289b7b8...,0xC1a1D956311AE0809e52b5F9873E0db627fb2D9A,1648933753,"[822.970834554, 0]",822,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot


In [6]:
# make date a pandas date, parse epoch timestamp type 's'
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0x586056a5f0f05b05b674b194b0c4709f85c2b09a5ab3...,0x46e7BC113E1f0748ddd15B49975c9dDD8E348DD2,2022-04-02 22:33:15,"[554.508218775, 1e-09]",554,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
1,0x3f0bf0cdbad6378da031424e8c4011626234f6797281...,0x9A0C4a2847A49769a2361c3fA5c9FC19b239BC1A,2022-04-02 22:24:10,"[40.978659197, 0]",40,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
2,0x0d8449d02c52710d94165c527498f9a1bba3e28c998c...,0x7934746C0B363c395Ec01e6eEdDC151f21A5d4b4,2022-04-02 21:57:44,"[9.194e-06, 2000.111080317]",2000,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
3,QmWGfUi7KdkqN72m3peuHfLJ8WzLXtFWnW7bWkGKrRa9ii,0xa2908B6212A1F093fF94DC2Cd06F5Df7a711A436,2022-04-02 21:13:59,"[5.9299e-05, 6.71957472]",6,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot
4,0xfd3b093c6b4b89396dba080eeb8c7345a0af3289b7b8...,0xC1a1D956311AE0809e52b5F9873E0db627fb2D9A,2022-04-02 21:09:13,"[822.970834554, 0]",822,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,snapshot


In [7]:
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'vote_id', 'voter', 'date', 'choice', 'weight']]

In [8]:
df.head()

,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,snapshot,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,0x586056a5f0f05b05b674b194b0c4709f85c2b09a5ab3...,0x46e7BC113E1f0748ddd15B49975c9dDD8E348DD2,2022-04-02 22:33:15,"[554.508218775, 1e-09]",554
1,snapshot,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,0x3f0bf0cdbad6378da031424e8c4011626234f6797281...,0x9A0C4a2847A49769a2361c3fA5c9FC19b239BC1A,2022-04-02 22:24:10,"[40.978659197, 0]",40
2,snapshot,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,0x0d8449d02c52710d94165c527498f9a1bba3e28c998c...,0x7934746C0B363c395Ec01e6eEdDC151f21A5d4b4,2022-04-02 21:57:44,"[9.194e-06, 2000.111080317]",2000
3,snapshot,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,QmWGfUi7KdkqN72m3peuHfLJ8WzLXtFWnW7bWkGKrRa9ii,0xa2908B6212A1F093fF94DC2Cd06F5Df7a711A436,2022-04-02 21:13:59,"[5.9299e-05, 6.71957472]",6
4,snapshot,saynotorug.eth,0x5e2e98efcb2938c860417eb25289b960d44dacff9807...,0xfd3b093c6b4b89396dba080eeb8c7345a0af3289b7b8...,0xC1a1D956311AE0809e52b5F9873E0db627fb2D9A,2022-04-02 21:09:13,"[822.970834554, 0]",822


In [9]:
# save
df.to_csv('snapshot_votes_small_proposals.csv', index=False)